### Section 0: Loading packages and testing earth engine connection

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import ee
import earthlib as eli
import folium

In [ ]:
%reload_ext ccblc

In [ ]:
# authenticate earth engine api access (not required on every run)
ee.Authenticate()

In [3]:
# initialize earth engine api access (required on every run)
ee.Initialize()

In [4]:
# test whether the initialization was successful by printing the elevation of Mount Everest
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


### Section 1: unmixing one image

In [5]:
# select the sensor to use
sensor = 'Landsat8'
collectionName = eli.getCollection(sensor)
bands = eli.getBands(sensor)
scaler = eli.getScaler(sensor)
maskFunction = eli.Mask.bySensor(sensor)
visBands = ["B6", "B5", "B4"]
#visBands = ["sur_refl_b06", "sur_refl_b05", "sur_refl_b01"]
#visBands = ["B11", "B5", "B4"]

# set the dates
startDate = "2020-03-01";
endDate = "2020-08-01";

# get a land mask to use
land = ee.Image("UMD/hansen/global_forest_change_2019_v1_7").select(["datamask"]).eq(1)

# create and filter the image collection
collection = ee.ImageCollection(collectionName).filterDate(startDate, endDate).map(maskFunction).select(bands)

In [7]:
# create a composite
sr = collection.median().multiply(scaler).updateMask(land)

# map it
folium.Map.addLayer = eli.Map.addLayer

# Set visualization parameters.
visParams = {
  'min': 0,
  'max': 0.5,
  'bands': visBands
}

# Create a folium map object.
Map = folium.Map(location=[37.5, -118], zoom_start=6, height=500)

# Add the image to the map object.
Map.addLayer(sr, visParams, sensor)

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
display(Map)

now we'll actually unmix the image using the first spectra from each bundle

In [11]:
# initialize the spectral bundle selection
eli.Unmix.setSensor(sensor, 30, bands)

# unmix the thing
unmixed = eli.Unmix.SVN(sr).updateMask(land)
#unmixed = collection.map(ccblc.Unmix.SVN).mean()

# map it
visParams = {'min': [0, 0.1, 0], 'max': [0.8, 1, 0.8]}
Map = folium.Map(location=[37.5, -119], zoom_start=7, height=500)
Map.addLayer(unmixed, visParams, 'unmixed')
Map.add_child(folium.LayerControl())
display(Map)

In [ ]:
xy = ee.Geometry.Point([37.5, -119])
#vals = unmixed.sample(xy, 30).select(["sum"]).first().getInfo()
#print(vals)
print(unmixed.getInfo())

In [12]:
# check out the modeled reflectance

# Set visualization parameters.
visParams = {
  'min': 0,
  'max': 0.35,
  'bands': visBands
}

visUnmixed = {
  'min': 0,
  'max': 0.35,
  'bands': ["RMSE"]
}

# Create a folium map object.
Map = folium.Map(location=[37.5, -118], zoom_start=6, height=500)

# Add the image to the map object.
Map.addLayer(sr, visParams, sensor)
Map.addLayer(unmixed, visUnmixed, "unmixed")

# Add a layer control panel to the map.
Map.add_child(folium.LayerControl())

# Display the map.
display(Map)